In [ ]:
import numpy as np 
import pandas as pd 
import keras.losses
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from tensorflow.python.keras.utils.np_utils import to_categorical
import re
from keras.backend import clear_session

In [ ]:
data = pd.read_csv('train.csv')
data =data.dropna()
print(data)

In [ ]:
data.isnull().sum()

In [ ]:
data.head() 

In [ ]:
data = data[['selected_text','sentiment']]
print(data)

In [ ]:
data = data[data.sentiment != "Neutral"]
data['selected_text'] = data['selected_text'].astype(str).apply(lambda x: x.lower())
data['selected_text'] = data['selected_text'].apply((lambda x:re.sub('[^a-zA-z0-9\s]','',x)))

 
print(data[ data['sentiment']=='positive'].size)
print(data[ data['sentiment']=='negative'].size)

for idx,row in data.iterrows():
  row[0]=row[0].replace('rt',' ')
max_features = 500
tokenizer = Tokenizer(num_words = max_features,split=' ')
tokenizer.fit_on_texts(data['selected_text'].values)
x=tokenizer.texts_to_sequences(data['selected_text'].values)
x=pad_sequences(x)

In [ ]:
y=pd.get_dummies(data['sentiment']).values

x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.1,random_state=42)
clear_session()
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)


In [ ]:
embed_dim = 128
lstm_out = 196

model = Sequential()

model.add(Embedding(max_features,embed_dim,input_length=x.shape[1]))

model.add(SpatialDropout1D(0.4))

model.add(LSTM(lstm_out,dropout=0.5,recurrent_dropout=0.2))

model.add(Dense(3,activation='softmax'))

model.add(Dense(3,activation='softmax'))
#model.compile(loss='categorical_crossentropy', optimizer=SGD, metrics=['accuracy'])

model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
#model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()


In [ ]:
batch_size = 56
model.fit(x_train, y_train, epochs = 30, batch_size=batch_size, verbose = 2,validation_split=0.1)